In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow as tf

# Import and inspect data

## Import

In [ ]:
def import_data():
    """Import training and testing data"""
    
    # Import in pandas (np.genfromtxt is too slow)
    tr_pd = pd.read_csv('../week_2/data_train.csv', header=None)
    tst_pd = pd.read_csv('../week_2/data_test.csv', header=None)
    
    # Remove labels
    tr_data = tr_pd.loc[:, 1:].values
    tst_data = tst_pd.loc[:, 1:].values
    
    return tr_data, tst_data

In [ ]:
g_tr_data, g_tst_data = import_data()
g_tr_data = g_tr_data/255
g_tst_data = g_tst_data/255

## Inspect

In [ ]:
def inspect_number(data, img_nums):
    """Display image. img_nums should be entered as list."""
    
    num_images = len(img_nums)
    # Reshape datasets into pixel array
    data_rs = np.reshape(data, newshape=(data.shape[0], 28, 28))
    
    figsize = (10, 100)
    fig, axes = plt.subplots(1, num_images, figsize=figsize)
    for k in range(num_images):
        ax = axes[img_nums[k]]
        ax.imshow(data_rs[img_nums[k]], cmap='Greys')
        ax.set_axis_off()
    plt.show()

# Build MLP in Tensorflow

In [ ]:
class AutoEncoder():
    """Autoencoder. For most examples, self.inp=self.tgt, but these are considered 
    independent variables to make extensions more straightforward."""
    
    def __init__(self, num_HL_nodes, learn_param):
        self.num_HL_nodes = num_HL_nodes    # Number of nodes in HL
        self.learn_param = learn_param
        
        self.inp = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='inputs')
        self.tgt = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='targets')
        self.sess = tf.Session()
        self._build_graph()
        self.sess.run(tf.initializers.global_variables())

    def _build_graph(self):
        """Build the neural network"""
        initializer = tf.glorot_uniform_initializer()
        
        # Encoder
        h = tf.layers.dense(self.inp, self.num_HL_nodes, kernel_initializer=initializer, 
                            activation=None, name='encoder_1')

        # Decoder
        self.out = tf.layers.dense(h, 784, kernel_initializer=initializer, 
                                   activation=None, name='decoder_1')  
    
        # Optimiser: mean squared error
        self.cost = tf.reduce_mean(tf.squared_difference(self.out, self.tgt))
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=self.learn_param).minimize(self.cost)
        
    def make_minibatch(self, data_inp, data_tgt, batch_size, batch_num):
        """Form a minibatch from the data"""
        llim = batch_num * batch_size
        rlim = (batch_num + 1) * batch_size
        return data_inp[llim:rlim], data_tgt[llim:rlim]
    
    def evaluate_AE_outputs(self, data_inp, data_tgt):
        """Run the autoencoder on some data and evaluate the mean squared error"""
        feed_dict = {self.inp: data_inp, self.tgt: data_tgt}
        outputs, cost = self.sess.run([self.out, self.cost], feed_dict=feed_dict)
        return outputs, cost

    def estimate_plots(self, data, img_nums):
        """Plot the autoencoder estimates over some images"""
        estimates = self.evaluate_AE_outputs(data[img_nums], data[img_nums])[0]
#        print(np.round(estimates[0, :10], 3))   # For debugging
        inspect_number(estimates, img_nums)
    
    def train_iteration(self, data_inp, data_tgt):
        """Do one training iteration over the input data"""
        feed_dict = {self.inp: data_inp, self.tgt: data_tgt}
        _, out = self.sess.run([self.optimizer, self.out], feed_dict=feed_dict)
        
    def train_full(self, tr_inp, tr_tgt, tst_inp, tst_tgt, 
                   batch_size, num_epochs, info=True, infoplots=False):
        """Train neural network and keep track of progress by (cross) validating"""
        
        # batch_size must divide num_tr_points
        num_tr_pts = tr_inp.shape[0]
        num_tst_pts = tst_inp.shape[0]
        num_batches = int(round(num_tr_pts / batch_size)) 
        
        # Train the MLP and keep track of the errors across training and testing datasets
        batch_nums, epochs, tr_costs, tst_costs = \
                np.array([]), np.array([]), np.array([]), np.array([])
        for epoch in range(num_epochs):
            for batch_num in range(num_batches):
                # Input proportion correctly identified across training and testing dataset
                if batch_num % 50 == 0:
#                if batch_num == 599:
                    _, tr_cost = self.evaluate_AE_outputs(tr_inp, tr_tgt)
                    _, tst_cost = self.evaluate_AE_outputs(tst_inp, tst_tgt)
                    if info is True:
                        print('Cost after {} batches: Tr {}, Tst {}'.format(
                            batch_num, tr_cost, tst_cost))
                    if infoplots is True: 
                        self.estimate_plots(g_tr_data, np.arange(10))
                    epochs = np.append(epochs, epoch)
                    batch_nums = np.append(batch_nums, batch_num)
                    tr_costs = np.append(tr_costs, tr_cost)
                    tst_costs = np.append(tst_costs, tst_cost)
                
                # Do the training over a minibatch
                mb_tr_inp, mb_tr_tgt = \
                    self.make_minibatch(tr_inp, tr_tgt, batch_size=batch_size, 
                                        batch_num=batch_num) 
                self.train_iteration(mb_tr_inp, mb_tr_tgt)    # Do single training iteration
                
        costs = pd.DataFrame(np.vstack((epochs, batch_nums, tr_costs, tst_costs)).T, 
                             columns = ['epoch', 'batch', 'train', 'test'])
        return costs

# Train model and evaluate

In [ ]:
def train_AutoEncoder(tr_data, tst_data, num_HL_nodes, num_epochs):
    model = AutoEncoder(num_HL_nodes=num_HL_nodes, learn_param=0.01)
    costs = model.train_full(tr_data, tr_data, tst_data, tst_data, 100, num_epochs, 
                                 info=False, infoplots=True)
#    model.estimate_plots(tr_data, np.arange(10))    # q: What's going on here?

In [ ]:
tf.reset_default_graph()
train_AutoEncoder(g_tr_data, g_tst_data, num_HL_nodes=32, num_epochs=1)

In [ ]:
inspect_number(g_tr_data, np.arange(10))
for num_HL_nodes in [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]:
    tf.reset_default_graph()
    train_AutoEncoder(g_tr_data, g_tst_data, num_HL_nodes=num_HL_nodes, num_epochs=1)

In [ ]:
"""Important results:

                        1 hidden   2 hidden
                        layer      layers
                        (128)      (128, 64)
                        
Number of parameters:   101770     109386
Time taken (5 epochs):  57.8s      59.5s
Final training error:   0.971      0.956
Final testing error:    0.962      0.948

"""

In [ ]:
"""Plot training and testing errors"""

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
errors_1HL = pd.read_csv('1HL.csv', index_col=[0])
errors_2HL = pd.read_csv('2HL.csv', index_col=[0])

X = (np.arange(60) + 1) / 12
tr_1HL = errors_1HL.loc[:, 'training']
tst_1HL = errors_1HL.loc[:, 'testing']
tr_2HL = errors_2HL.loc[:, 'training']
tst_2HL = errors_2HL.loc[:, 'testing']
ax1.plot(X, tr_1HL)
ax1.plot(X, tst_1HL)
ax2.plot(X, tr_2HL)
ax2.plot(X, tst_2HL)
ax1.set_xlabel('Runs through dataset')
ax2.set_xlabel('Runs through dataset')
ax1.set_ylabel('Proportion correct')
ax1.set_title('1 Hidden layer')
ax2.set_title('2 Hidden layers')
ax1.legend()
ax2.legend()
plt.savefig('Errors')